# **Telecom Churn Case Study**

## Business problem overview

The objective of this project is to help a prominent telecom company reduce customer churn by predicting which customers are most likely to leave. To achieve this goal, we will analyze data at the customer level and create predictive models that can identify individuals who are at a high risk of churning. Our focus will be on retaining profitable customers who are critical to the company's success. Additionally, we aim to identify the key factors that contribute to customer churn, so that the company can take appropriate measures to address them.

In [5]:
# Importing the required libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor